# Advent of code 2024
## Challenge 12

## Part 1
### https://adventofcode.com/2024/day/12

In [1]:
class Position:
  def __init__(self, value):
    self.value = value
    self.visited = False

# Reading the data input file
input_file = open("challenge_12_input.txt", "r")
maze = []
start_locations = []
maze_with_marker = []

# turning the input data into a list of lists of strings
for line in input_file:
    input_data = list(line.strip())
    maze.append(input_data)

# turning the list of lists of strings into a list of lists of objects with two values:
# the string value from the input, and a bool mentioning if it is visited or not
for line in maze:
    temp_list = []
    for value in line:
        temp_list.append(Position(value))
    maze_with_marker.append(temp_list.copy())

In [2]:
# The directions to take steps
directions = [(0,1),(1,0),(-1,0),(0,-1)]
# Maze boudaries to avoid going out of bounds in the loop
maze_boundaries = [len(maze_with_marker),len(maze_with_marker[1])]

# Is the position within bounds
def is_valid(position, maze_limits, maze):
    return 0 <= position[0] < maze_limits[0] and 0 <= position[1] < maze_limits[1]

# This is the walking function, it is recursive
def find_path(position, maze, maze_limits, found_destinations):  
    
    # Each position inserted as position is a position that can be walked on and is therefore automatically added to
    # a region list, and the position is marked as stepped on
    found_destinations.append(position)
    maze[position[0]][position[1]].visited = True
    
    # We take a step in every possible direction if the next position has the same value as the previous, meaning it is in the 
    # same region, and it is not already stepped on, the step is allowed
    for i in range(4):
        # Calculate the next position according to direction
        next_position = [position[0] + directions[i][0], position[1] + directions[i][1]]
        
        if is_valid(next_position, maze_boundaries, maze) and maze[next_position[0]][next_position[1]].visited == False and maze[next_position[0]][next_position[1]].value == maze[position[0]][position[1]].value:
            # If the new position is valid, we recurse
            find_path(next_position, maze, maze_limits, found_destinations)
    return

# The perimeter value of a position is calculated by the amount of neighbor it has
# if there is a neighbor, it means that it doesn't touch the edge on this side, and thus a value of one is removed
def find_perimeter_value(position,region):
    perimeter_value = 4
    for i in range(4):
        neighbor = [position[0] + directions[i][0], position[1] + directions[i][1]]
        if neighbor in region:
            perimeter_value -= 1
    return perimeter_value
    

region = []
region_list = []

# We find all of our regions in this loop
for main_index,maze_path in enumerate(maze_with_marker):
    for sub_index,position in enumerate(maze_path):
        if position.visited == False:
            find_path([main_index,sub_index], maze_with_marker, maze_boundaries, region)
            region_list.append(region.copy())
            region = []

total_price = 0
 
# We calculate the perimeter value of each region, multiplied by the number of positions
# according to specifications, and accumulate that result to obtain the total price.
for region in region_list:
    perimeter = 0
    for position in region:
        perimeter += find_perimeter_value(position,region)
    total_price += perimeter * len(region)
    
print(total_price)

1450422


## Part 2

This part of the challenge was a real challenge for me. It was quite challenging to find a logic that could find the sides. After many hours of pondering, I came to an algorithm without outside help. But before starting to code, I laid down the plan under. I left it there for posterity.

In [3]:
# Here, we just need to extract all the edges. These are the positions with directions. This will make for unique edges

# After that, all those edges need to be sorted

# There are 4 possibilities for the 4 directions, which then have to be further sorted by the main index for the top and down
# direction, and the nested index for the left and right direction, this can be done with a dictionnary 

# Once this is found, we just need to sort the varying lists from smallest to biggest, we start with 1, and we don't add
# one until the upcoming number is bigger by more then 1 from the previous edge, which means the edges are not, we go through
# each lists of the shape to have the number of sides, which we then multiply by the length of the original list 
# we started with.

In [4]:
class Position:
  def __init__(self, value):
    self.value = value
    self.visited = False

input_file = open("challenge_12_input.txt", "r")
maze = []
start_locations = []
maze_with_marker = []

for line in input_file:
    input_data = list(line.strip())
    maze.append(input_data)

for line in maze:
    temp_list = []
    for value in line:
        temp_list.append(Position(value))
    maze_with_marker.append(temp_list.copy())

In [5]:
# The directions to take steps, I created a dictionnary this time to make it easier to follow
directions_dictionary = {
    'right': (0,1), 
    'left': (0,-1), 
    'top': (-1,0),
    'bottom':(1,0)
}

maze_boundaries = [len(maze_with_marker),len(maze_with_marker[1])]

def is_valid(position, maze_limits, maze):
    return 0 <= position[0] < maze_limits[0] and 0 <= position[1] < maze_limits[1]

def find_path(position, maze, maze_limits, found_destinations):  
    found_destinations.append(position)
    maze[position[0]][position[1]].visited = True
    
    for i in range(4):
        next_position = [position[0] + directions[i][0], position[1] + directions[i][1]]
        
        if is_valid(next_position, maze_boundaries, maze) and maze[next_position[0]][next_position[1]].visited == False and maze[next_position[0]][next_position[1]].value == maze[position[0]][position[1]].value:
            find_path(next_position, maze, maze_limits, found_destinations)
    return

# This function precisely locates all the edges of a region. It is done according to the plan laid down above
def find_edges(position,region,edges_dictionnary):
    if [position[0] + directions_dictionary['right'][0], position[1] + directions_dictionary['right'][1]] not in region:
        key = ('right',position[1])
        if key not in edges_dictionnary:
            edges_dictionnary[key] = []
        edges_dictionnary[key].append(position[0])
    if [position[0] + directions_dictionary['left'][0], position[1] + directions_dictionary['left'][1]] not in region:
        key = ('left',position[1])
        if key not in edges_dictionnary:
            edges_dictionnary[key] = []
        edges_dictionnary[key].append(position[0])
    if [position[0] + directions_dictionary['top'][0], position[1] + directions_dictionary['top'][1]] not in region:
        key = ('top',position[0])
        if key not in edges_dictionnary:
            edges_dictionnary[key] = []
        edges_dictionnary[key].append(position[1])
    if [position[0] + directions_dictionary['bottom'][0], position[1] + directions_dictionary['bottom'][1]] not in region:
        key = ('bottom',position[0])
        if key not in edges_dictionnary:
            edges_dictionnary[key] = []
        edges_dictionnary[key].append(position[1])
    
region = []
region_list = []

for main_index,maze_path in enumerate(maze_with_marker):
    for sub_index,position in enumerate(maze_path):
        if position.visited == False:
            find_path([main_index,sub_index], maze_with_marker, maze_boundaries, region)
            region_list.append(region.copy())
            region = []

total_cost = 0

# For each found region
for region in region_list:
    edges_sorted = {}
    number_of_sides = 0
    # We extract the edges
    for position in region:
        find_edges(position,region,edges_sorted)
    # Out of all the edges, we calculate the sides
    for key in edges_sorted:
        # For each key in the map, it means that there is at least one side
        number_of_sides += 1
        if len(edges_sorted[key]) > 1:
            # if the list has more then one edge, we sort them. Edges that are next to each other
            # form one side, so we only add another side when an edge that is no longer next to the
            # previous one is found, that is if its distance with the previous edge is bigger then 1
            edges_sorted[key] = sorted(edges_sorted[key])
            for i in range(len(edges_sorted[key]) - 1):
                if edges_sorted[key][i + 1] - edges_sorted[key][i] > 1:
                    number_of_sides += 1
    # lastly, we multiply the number of sides by the surface of the region, which is calculated as 1 per position
    # According to specifications
    total_cost += number_of_sides * len(region)

print(total_cost)

906606
